### Plot tTEM with Gamma

In [1]:
# Parameters
import pandas as pd
import numpy as np
import tTEM_toolbox as tt
from pathlib import Path
#from plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
workdir = Path(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test')
welllog = workdir.joinpath(r'Plot_with_well_log\Well_log.xlsx')
elevation = workdir.joinpath(r'well_Utah\usgs_water_elevation.csv')
ttemname_north = workdir.joinpath(r'Plot_with_well_log\PD1_I01_MOD.xyz')
ttemname_center = workdir.joinpath(r'Plot_with_well_log\PD22_I03_MOD.xyz')
ttem_lslake = workdir.joinpath(r'Plot_with_well_log\lsll_I05_MOD.xyz')
DOI = workdir.joinpath(r'Plot_with_well_log\DOID1_DOIStaE.xyz')
well_info = workdir.joinpath(r'well_Utah\USGSdownload\NWISMapperExport.xlsx')
gamma_file_path = r"C:\Users\jldz9\PycharmProjects\tTEM_toolbox\data\Parowan water table.xlsx"
location = r"C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\location.csv"
colorRes = [[0, 'rgb(0,0,190)'],
            [1 / 16, 'rgb(0,75,220)'],
            [2 / 16, 'rgb(0,150,235)'],
            [3 / 16, 'rgb(0,200,255)'],
            [4 / 16, 'rgb(80,240,255)'],
            [5 / 16, 'rgb(30,210,0)'],
            [6 / 16, 'rgb(180,255,30)'],
            [7 / 16, 'rgb(255,255,0)'],
            [8 / 16, 'rgb(255,195,0)'],
            [9 / 16, 'rgb(255,115,0)'],
            [10 / 16, 'rgb(255,0,0)'],
            [11 / 16, 'rgb(255,0,120)'],
            [12 / 16, 'rgb(140,40,180)'],
            [13 / 16, 'rgb(165,70,220)'],
            [14 / 16, 'rgb(195,130,240)'],
            [15 / 16, 'rgb(230,155,240)'],
            [1, 'rgb(230,155,255)']]

In [2]:
gamma_elevation = pd.read_csv(location)
bradshaw_farm_elevation = gamma_elevation[gamma_elevation['name'] == 'Bradshaw Farm']
bradshaw_farms = pd.read_excel(r'C:\Users\jldz9\PycharmProjects\tTEM_toolbox\data\Parowan water table.xlsx',sheet_name='bradshaw farms down',skiprows=[1])
bradshaw_farms.rename(columns=lambda x: str(x).strip(),inplace=True)
bradshaw_farms_clean = bradshaw_farms[~(bradshaw_farms['COUNT'] == -999)]

In [3]:
from pyproj import Transformer
x_ori = bradshaw_farm_elevation.iloc[0]["X"]
y_ori = bradshaw_farm_elevation.iloc[0]["Y"]
transformer = Transformer.from_crs('epsg:4326', 'epsg:32612')  # WGS84-->UTM12N
x, y = transformer.transform(y_ori, x_ori)
bradshaw_farms_clean.loc[:, "X"] = x
bradshaw_farms_clean.loc[:, "Y"] = y
bradshaw_farms_clean.loc[:, "Z"] = bradshaw_farm_elevation.loc[:, 'Z'].values[0]
bradshaw_farms_clean.loc[:, "Elevation"] = bradshaw_farms_clean.loc[:, "Z"].sub(bradshaw_farms_clean.loc[:, "Depth"])

In [5]:
ttem_n = tt.main.ProcessTTEM(ttem_path=[ttemname_north],
                                   welllog=welllog,
                                   DOI_path=DOI,
                                   layer_exclude=[],
                                    line_exclude=[])


Applying DOI |                                | 68/7968

Exclude layer []
Exclude line []
No point was excluded


Applying DOI |################################| 7968/7968


In [6]:
ttem_north = ttem_n.data()

In [ ]:
ttem_north

In [7]:
import pathlib
def get_distance(group1, group2):
    dis = np.sqrt((group1[0] - group2[0]) ** 2 + (group1[1] - group2[1]) ** 2)
    return dis
def select_closest_gamma(ttem_data, gamma_data,distance=500):
    UTMX_gamma = gamma_data["X"].iloc[0]
    UTMY_gamma = gamma_data["Y"].iloc[0]
    ttem_data["distance"] = np.sqrt((ttem_data["UTMX"] - UTMX_gamma) ** 2 +
                                    (ttem_data["UTMY"] - UTMY_gamma) ** 2)
    #match base on distance equation l = sqrt((x1-x2)**2 + (y1-y2)**2)
    df_closest = ttem_data[ttem_data["distance"] == ttem_data.distance.min()].reset_index(drop=True)
    df_closest = df_closest[df_closest["distance"] <= float(distance)]
    return df_closest

In [8]:
df_closest = select_closest_gamma(ttem_north,bradshaw_farms_clean)

In [ ]:
import importlib
importlib.reload(tt.plot)
fig = tt.plot.res_1d_plot(df_closest)

ID :2628


In [13]:
df_closest.Resistivity.max()

91.9